In [12]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import torch
klasor_yolu = 'img_train/' 
dosya_isimleri = [dosya for dosya in os.listdir(klasor_yolu) if dosya.endswith('.jpg')]

hedef_boyut = (106, 106)

data = []
for dosya in dosya_isimleri:
    tam_yol = os.path.join(klasor_yolu, dosya)
    with Image.open(tam_yol) as img:
        img = img.convert('L').resize(hedef_boyut)  
        numpy_dizi = np.array(img).flatten()  
        data.append(numpy_dizi)

sonuc_matrisi = np.vstack(data)

csv_dosya_yolu = 'train_sol.csv' 
csv_matrisi = pd.read_csv(csv_dosya_yolu).values

csv_matrisi = csv_matrisi[:, 1:]

data_train = np.hstack((sonuc_matrisi/255, csv_matrisi)) 


In [13]:
data_train.max()

1.0

In [3]:
klasor_yolu = 'img_test/'  
dosya_isimleri = [dosya for dosya in os.listdir(klasor_yolu) if dosya.endswith('.jpg')]

hedef_boyut = (106, 106)

data = []
for dosya in dosya_isimleri:
    tam_yol = os.path.join(klasor_yolu, dosya)
    with Image.open(tam_yol) as img:
        img = img.convert('L').resize(hedef_boyut)  
        numpy_dizi = np.array(img).flatten()  
        data.append(numpy_dizi)

sonuc_matrisi = np.vstack(data)

csv_dosya_yolu = 'train_sol.csv' 
csv_matrisi = pd.read_csv(csv_dosya_yolu).values

csv_matrisi = csv_matrisi[:, 1:]

data_test = np.hstack((sonuc_matrisi, csv_matrisi))

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 12316 and the array at index 1 has size 49262

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class SimpleNeuralNetwork(nn.Module):
    def __init__(self):
        super(SimpleNeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(11236, 100).to(device)
        self.layer2 = nn.Linear(100, 100).to(device)
        self.output_layer = nn.Linear(100, 37).to(device)
        self.relu = nn.ReLU().to(device)

    def forward(self, x):
        x = x.to(device)
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.output_layer(x)
        return x

net = SimpleNeuralNetwork().to(device)


In [14]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
torch.cuda.empty_cache()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_train_tensor = torch.from_numpy(data_train).float().to(device)

features = data_train_tensor[:, :-37]
labels = data_train_tensor[:, -37:]
train_dataset = torch.utils.data.TensorDataset(features, labels)

num_epochs =  32

net = SimpleNeuralNetwork().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

def get_data_loader(data, batch_size, shuffle=False):
    dataset = torch.utils.data.TensorDataset(data[:, :-37], data[:, -37:].long())
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
 
chunk_size = 1000  
data_chunks = [data_train_tensor[i:i + chunk_size] for i in range(0, data_train_tensor.size(0), chunk_size)]

for epoch in range(num_epochs):
    for data_chunk in data_chunks:
        train_loader = get_data_loader(data_chunk, batch_size=16)

        for batch in train_loader:
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device).float()

            outputs = net(inputs).float()


            loss = loss_function(outputs, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            torch.cuda.empty_cache()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Chunk Loss: {loss.item():.8f}')


Epoch [1/32], Chunk Loss: 0.00061207
Epoch [1/32], Chunk Loss: -0.00000000
Epoch [1/32], Chunk Loss: -0.00000000
Epoch [1/32], Chunk Loss: -0.00000000
Epoch [1/32], Chunk Loss: 0.04969913
Epoch [1/32], Chunk Loss: -0.00000000
Epoch [1/32], Chunk Loss: -0.00000000
Epoch [1/32], Chunk Loss: 0.02866882
Epoch [1/32], Chunk Loss: 1.06962943
Epoch [1/32], Chunk Loss: 0.02646487
Epoch [1/32], Chunk Loss: -0.00000000
Epoch [1/32], Chunk Loss: -0.00000000
Epoch [1/32], Chunk Loss: -0.00000000
Epoch [1/32], Chunk Loss: 0.04710881
Epoch [1/32], Chunk Loss: 0.02767462
Epoch [1/32], Chunk Loss: 0.07397116
Epoch [1/32], Chunk Loss: 0.07203469
Epoch [1/32], Chunk Loss: 1.06302142
Epoch [1/32], Chunk Loss: -0.00000000
Epoch [1/32], Chunk Loss: 1.12480426
Epoch [1/32], Chunk Loss: -0.00000000
Epoch [1/32], Chunk Loss: 0.02808027
Epoch [1/32], Chunk Loss: -0.00000000
Epoch [1/32], Chunk Loss: 0.02487812
Epoch [1/32], Chunk Loss: -0.00000000
Epoch [1/32], Chunk Loss: -0.00000000
Epoch [1/32], Chunk Loss:

KeyboardInterrupt: 